# Visualizing Large Social Networks


### Instructions

Download <a href="Add Link Here/facebook_network"> Sample Facebook Network Data </a> from CS Git Community Folder: <b>DS_Course</b>

Environment: <b>Python 3x</b>

Install required Python libraries:<br>

<b>conda install plotly</b><br>
<b>conda install networkx</b>

Download python-louvain library: <a href="https://pypi.python.org/pypi/python-louvain/0.3"> python-louvai </a>

Open terminal and browse to downloaded folder. Install Library: <b>python setup.py install<b><br>

<a href="https://plot.ly/python/network-graphs"> Reference </a>

In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
import networkx as nx

### Get Node Positions
Store position as node attribute data for random_geometric_graph and find node near center (0.5, 0.5)

In [2]:
G = nx.read_edgelist("facebook_network_2500.txt", create_using = nx.Graph(), nodetype = int)
pos = nx.spring_layout(G)

for i in G:
    G.node[i] = {'pos': pos[i]}

for key in pos:
    pos[key] = list(pos[key])

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

p=nx.single_source_shortest_path_length(G,ncenter) 

### Create Edges

Add edges as disconnected lines in a single trace and nodes as a scatter trace

In [3]:
edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='Portland',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)


### Color Node Points

Because your friends can be broken down into different groups, we may think if we could identify these different communities in our social network. The criteria for finding good communities is similar to that for finding good clusters. Good communities should have a high number of intra-community edges. While there is no community detection method in NetworkX, a library "community" is built on top of NetworkX.<br>

<b>Note:</b> Avoid the below step if you are not looking to give a color dimension to the Graph

In [4]:
import community

In [5]:
parts = community.best_partition(G)
values = [parts.get(node) for node in G.nodes()]

for node, adjacencies in enumerate(G.adjacency_list()):
    node_info = '# of connections: '+ str(len(adjacencies))
    node_trace['text'].append(node_info)

node_trace['marker']['color'] = values

### Create Network Graph

In [6]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Facebook Connections Network Graph<br>',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig, filename='networkx')